In [1]:
from ultralytics import YOLO

In [ ]:
model = YOLO('idiot') #specs as arg

FileNotFoundError: 'idiot' does not exist

> /home/rakshat/.virtualenvs/study/lib/python3.10/site-packages/ultralytics/utils/checks.py(520)check_file()
    518         files = glob.glob(str(ROOT / "**" / file), recursive=True) or glob.glob(str(ROOT.parent / file))  # find file
    519         if not files and hard:
--> 520             raise FileNotFoundError(f"'{file}' does not exist")
    521         elif len(files) > 1 and hard:
    522             raise FileNotFoundError(f"Multiple files match '{file}', specify exact path: {files}")

